In [1]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

In [3]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    auctions_df = pd.read_sql(sql_query, con=db_con)

auctions_df

,id,eventTimestamp,unitDisplayType,brandName,bundleId,countryCode,deviceId,osAndVersion,bidFloorPrice,sentPrice
0,1,1657758857892,rewarded,Generic,com.loop.match3d,US,6a0a94554cf,Android-4.0,0.05,0.06
1,2,1657210707978,interstitial,Generic,com.loop.match3d,ZA,6a0b0e59f45,Android-4.0,0.01,0.16
2,3,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,BR,6a0fa820c46,Android-10.0,0.05,0.06
3,4,1657386816882,interstitial,Huawei,com.tintash.nailsalon,IQ,6a142bdbea2,Android-9.0,0.01,0.05
4,5,1657211600823,banner,Generic,com.tilegarden.match3,US,6a16943a771,Android-4.0,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...
83163,83164,1657731605063,banner,Generic,com.tilegarden.match3,NZ,80edf5837e8,Android-4.0,0.01,0.02
83164,83165,1657657131101,interstitial,Apple,1436213906,VN,80ee6a5eebf,iOS-13.6,0.01,0.90
83165,83166,1657156070854,rewarded,Generic,com.loop.match3d,US,80efe531afc,Android-4.0,0.05,0.69
83166,83167,1657659603568,banner,Apple,1502447854,US,80f02dd397f,iOS-15.0,0.01,0.10


## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [4]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        app_vectors_df = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~app_vectors_df["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

app_vectors_df


,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683534,0.6250089,-0.120355874,0.21742316,..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112804,0.7446751,-0.58174485,0.9596781,0...."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176633,0.6059462,-1.1395651,0.076310374,-..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924532,1.0967331,-0.30831638,0.13220643,..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975613,0.46442544,-0.18019192,1.2617054,0...."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.88490516,0.65851974,-0.33866894,0.18323667,..."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.20005327,0.36083457,-0.13792287,0.34308684,..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112476,0.5128062,-0.056304686,1.2819327,0...."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.7309438,0.41121688,-0.6196016,0.53501374,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.20005342,0.36083454,-0.1379227,0.34308654,-..."


We can use the `<=>` operator to run vector search within the database

In [5]:

vec = json.loads(app_vectors_df.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

Embedding size: 1024


,bundleId
0,com.loop.match3d
1,com.loop.match3d
2,1502447854
3,1502447854
4,1529614832
5,1529614832
6,com.tilegarden.match3
7,com.tilegarden.match3
8,com.kamilbilge.ropesavior3d
9,com.kamilbilge.ropesavior3d


# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Sample a fraction of the auctions dataFrame to manage memory usage
auctions_sampled_df = auctions_df.sample(frac=0.05, random_state=1)
auctions_sampled_df['sentPrice'] = pd.to_numeric(auctions_sampled_df['sentPrice'], errors='coerce')

# Convert the stringified embeddings into actual numpy arrays
if isinstance(app_vectors_df['embedding'].iloc[0], str):
    app_vectors_df['embedding'] = app_vectors_df['embedding'].apply(lambda x: np.array(json.loads(x)))

# Calculate cosine similarity for the first 10% of the app embeddings to the entire set
similarity_matrix = cosine_similarity(
    np.stack(app_vectors_df['embedding'].iloc[:int(len(app_vectors_df) * 0.1)]),
    np.stack(app_vectors_df['embedding'])
)

# Create a DataFrame with the app pairs and similarity scores
similarity_pairs = pd.DataFrame(
    [(i, j, similarity_matrix[i, j])
     for i in range(len(similarity_matrix))
     for j in range(len(similarity_matrix[i]))
     if i != j],
    columns=['app_index_1', 'app_index_2', 'similarity_score']
)

# Map the indices to bundleIds
similarity_pairs['bundleId_1'] = app_vectors_df.iloc[similarity_pairs['app_index_1']]['bundleId'].values
similarity_pairs['bundleId_2'] = app_vectors_df.iloc[similarity_pairs['app_index_2']]['bundleId'].values

# Merge with auction prices for bundleId_1
merged_df = similarity_pairs.merge(
    auctions_sampled_df[['bundleId', 'sentPrice']],
    left_on='bundleId_1',
    right_on='bundleId',
    how='left'
)
merged_df = merged_df.drop('bundleId', axis=1)

# Merge with auction prices for bundleId_2
merged_df = merged_df.merge(
    auctions_sampled_df[['bundleId', 'sentPrice']],
    left_on='bundleId_2',
    right_on='bundleId',
    how='left',
    suffixes=('_1', '_2')
)
merged_df = merged_df.drop('bundleId', axis=1)

# Calculate the absolute difference in sentPrice between each app pair
merged_df['price_difference'] = np.abs(merged_df['sentPrice_1'] - merged_df['sentPrice_2'])

# Calculate the correlation between similarity score and price difference
correlation = merged_df[['similarity_score', 'price_difference']].corr()

correlation


,similarity_score,price_difference
similarity_score,1.00000,0.02785
price_difference,0.02785,1.00000


#### It seems that app description doesn't make a good correlation with the app asked price. 
#### However, correlation is not the case, we're looking for causation so let's check some other different subsets of the auctions data using bootstrapping

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import norm

# Define the number of bootstrap samples to take
n_bootstrap_samples = 10

# Placeholder for storing bootstrap correlation coefficients
bootstrap_correlations = []

for _ in range(n_bootstrap_samples):
    # Sample a fraction of the auctions DataFrame
    auctions_sampled_df = auctions_df.sample(frac=0.05)
    auctions_sampled_df['sentPrice'] = pd.to_numeric(auctions_sampled_df['sentPrice'], errors='coerce')

    # Assuming embeddings are in the correct format
    # Calculate cosine similarity for a subset of the app embeddings
    subset_size = int(len(app_vectors_df) * 0.1)  # Calculate 10% of the dataset size
    app_embeddings_subset = np.stack(app_vectors_df['embedding'].iloc[:subset_size].tolist())
    similarity_matrix = cosine_similarity(app_embeddings_subset, np.stack(app_vectors_df['embedding'].tolist()))

    # Create a DataFrame with app pairs and similarity scores
    similarity_pairs = pd.DataFrame(
        [(i, j, similarity_matrix[i, j])
         for i in range(subset_size)
         for j in range(len(app_vectors_df))
         if i != j],
        columns=['app_index_1', 'app_index_2', 'similarity_score']
    )
    
    # Map the indices to bundleIds
    similarity_pairs['bundleId_1'] = app_vectors_df.iloc[similarity_pairs['app_index_1']]['bundleId'].values
    similarity_pairs['bundleId_2'] = app_vectors_df.iloc[similarity_pairs['app_index_2']]['bundleId'].values

    # Merge with auction prices for bundleId_1
    merged_df = similarity_pairs.merge(
        auctions_sampled_df[['bundleId', 'sentPrice']],
        left_on='bundleId_1',
        right_on='bundleId',
        how='left'
    )
    
    # Drop the 'bundleId' column to avoid the MergeError
    merged_df.drop(columns=['bundleId'], inplace=True)
    
    # Merge with auction prices for bundleId_2
    merged_df = merged_df.merge(
        auctions_sampled_df[['bundleId', 'sentPrice']],
        left_on='bundleId_2',
        right_on='bundleId',
        how='left',
        suffixes=('_1', '_2')
    )
    
    # Again drop the 'bundleId' column to avoid the MergeError
    merged_df.drop(columns=['bundleId'], inplace=True)

    # Calculate the absolute difference in sentPrice between each app pair
    merged_df['price_difference'] = np.abs(merged_df['sentPrice_1'] - merged_df['sentPrice_2'])

    # Calculate the correlation between similarity score and price difference
    correlation = merged_df[['similarity_score', 'price_difference']].corr().iloc[0, 1]
    
    # Store the correlation coefficient
    bootstrap_correlations.append(correlation)

# Convert the list of correlations to a numpy array for analysis
bootstrap_correlations = np.array(bootstrap_correlations)

# Calculate the mean and 95% confidence interval for the correlation coefficients
mean_correlation = np.mean(bootstrap_correlations)
conf_int = np.percentile(bootstrap_correlations, [2.5, 97.5])

print(f"Mean Correlation: {mean_correlation}")
print(f"95% confidence interval for the correlation coefficient: {conf_int}")


Mean Correlation: 0.027163668915743656
95% confidence interval for the correlation coefficient: [0.01386328 0.0384598 ]


### Alright, wev'e airtightd it with few more samples - 
## 1.There is no causation between app description and asked auction price

## Now for the countrycode and OS

In [12]:
# Mean sentPrice by CountryCode
mean_price_by_country = auctions_df.groupby('countryCode')['sentPrice'].mean().reset_index()

# Mean sentPrice by OS
mean_price_by_os = auctions_df.groupby('osAndVersion')['sentPrice'].mean().reset_index()

# Display the results
mean_price_by_country, mean_price_by_os

(    countryCode  sentPrice
 0            AD   0.110000
 1            AE   1.094737
 2            AF   0.246250
 3            AL   0.177632
 4            AM   0.235909
 ..          ...        ...
 166          VN   0.501848
 167          XK   0.200000
 168          ZA   0.692160
 169          ZM   0.090000
 170          ZW   0.156000
 
 [171 rows x 2 columns],
      osAndVersion  sentPrice
 0    Android-10.0   0.748694
 1    Android-11.0   0.955876
 2    Android-12.0   0.050000
 3     Android-4.0   1.753592
 4   Android-4.4.2   0.097778
 ..            ...        ...
 92       iOS-15.4   2.224300
 93     iOS-15.4.1   2.072420
 94       iOS-15.5   2.546158
 95       iOS-15.6   2.102500
 96       iOS-16.0   4.828333
 
 [97 rows x 2 columns])

In [13]:
import scipy.stats as stats

# Perform ANOVA for CountryCode
anova_country = stats.f_oneway(
    *[group['sentPrice'].dropna().values for name, group in auctions_df.groupby('countryCode')]
)

# Perform ANOVA for OS
anova_os = stats.f_oneway(
    *[group['sentPrice'].dropna().values for name, group in auctions_df.groupby('osAndVersion')]
)

# Display the ANOVA results
anova_country.pvalue, anova_os.pvalue


(0.0, 7.071723968614428e-126)

### We USED ANOVA to check for any correlation between the country or OS and got a very low P-Value.
## 2. Hence, There is no causation or correlation between the two attributes to the predicted price